<a href="https://colab.research.google.com/github/JDM-1609/Statistical-Process-Control-in-Injection-Machines/blob/main/SPC_Analysis_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EXTRACCIÓN DE ESTADÍSTICOS INICIALES

**SEPARACIÓN MANUAL DE LAS SECCIONES DEL CSV**

El siguiente código separa en DataFrames diferentes cada una de las secciones del CSV (Info. Contextual, estadísticos y datos crudos) de manera manual, dado que se le debe especificar los indices en donde comienza y termina cada sección. Este código tiene como limitante que si la distribución de los datos del CSV cambia el codigo se rompe (se tendrían que especificar los nuevos indices de inicio y fin de cada sección)

In [ ]:
import csv
from pathlib import Path
from io import StringIO
import pandas as pd
import numpy as np

# OPCIONAL: para visualización de los datos en el DF

#pd.set_option("display.max_columns", 20) # Controla cuántas columnas se muestran sin omisiones
pd.set_option("display.float_format", lambda v: f"{v:.4g}") # Muestra valores con hasta 4 cifras significativas

# Encoding de los archivos del ALS
ENCODING_ALS = "UTF-8"

In [ ]:
# Carga de datos
from google.colab import files

uploaded = files.upload()
RUTA_IN = next(iter(uploaded.keys()))
print("Archivo cargado:", RUTA_IN)

In [ ]:
# Indices start & end de cada sección
# SECCIONES
idx_info = 0           # Información contextual
idx_stats_start = 2    # Start Estadísticos
idx_stats_end = 21     # End Estadísticos
idx_raw_start = 23     # Datos crudos

In [ ]:
# FUNCIONES PARA PARSEAR SECCIONES

# Lee el CSV como strings y devuelve una lista donde cada elemento es una línea del CSV
def leer_lineas(ruta_csv: str, encoding: str = ENCODING_ALS):
    raw = Path(ruta_csv).read_text(encoding=encoding, errors="replace")
    lines = raw.splitlines()  # Este método siempre devuelve una lista
    return lines

# Devuelve la sección "Información Contextual" como un DF
def parse_info_contextual(lines, idx_info: int = 0) -> pd.DataFrame:
    line = lines[idx_info]  # Se toma únicamente la primera línea
    cells = next(csv.reader([line], delimiter=",", quotechar='"'))

    # Limpieza eliminando vacíos y el elemento "Valores reales"
    cleaned = []
    for c in cells:
        c = c.strip().strip('"')
        if not c:  # Vacío -> se omite
            continue
        if c.lower() == "valores reales":
            continue
        cleaned.append(c)

    claves  = cleaned[0::2]
    valores = cleaned[1::2]

    df_info = pd.DataFrame([valores], columns=claves)
    return df_info

# Devuelve la sección "Estadísticos" como un DF
def parse_estadisticos(lines, idx_start: int, idx_end: int) -> pd.DataFrame:

    # Se unen las líneas de la sección en un solo string tipo CSV
    block = "\n".join(lines[idx_start:idx_end])

    # Leemos este bloque como si fuera un CSV independiente
    df_stats = pd.read_csv(
        StringIO(block),
        sep=",",
        decimal=",",
        thousands=".",
        quotechar='"',
        encoding=ENCODING_ALS,
        engine="python",
        )
    # Los índices del df son los nombres de los estadísticos
    df_stats.index.name = "Estadísticos"

    # Ajuste de los encabezados de las columnas
    cols = list(df_stats.columns)
    if str(cols[0]).strip() == "":
      df_stats.columns = cols[1:] + cols[:1] # Se desplazan las columnas a la izq
      df_stats = df_stats.dropna(axis=1, how="all") # Elimina la columna vacía

    return df_stats

# Devuelve la sección "Datos Crudos" como un DF
def parse_datos_crudos(lines, idx_start: int) -> pd.DataFrame:

    # Se unen las líneas desde idx_start hasta final de los datos
    block = "\n".join(lines[idx_start:])

    df_raw = pd.read_csv(
        StringIO(block),
        sep=",",
        decimal=",",
        thousands=".",
        quotechar='"',
        encoding=ENCODING_ALS,
        engine="python",
    )
    df_raw.columns = [str(c).strip() for c in df_raw.columns]
    df_raw = df_raw.reset_index()

    cols = list(df_raw.columns)
    df_raw.columns = cols[1:] + cols[:1] # Se desplazan las columnas a la izq

    # 4. Eliminar columnas completamente vacías (NaN en todas las filas)
    df_raw = df_raw.dropna(axis=1, how="all")

    return df_raw


In [ ]:
# EJECUCIÓN

# Leer todas las líneas del archivo
lines = leer_lineas(RUTA_IN, encoding=ENCODING_ALS)
print(f"NÚMERO TOTAL DE LÍNEAS EN EL ARCHIVO: {len(lines)}\n")

# Información contextual
df_info = parse_info_contextual(lines, idx_info=idx_info)
print("INFORMACIÓN CONTEXTUAL:")
display(df_info)

# Estadísticos
df_stats = parse_estadisticos(lines, idx_start=idx_stats_start, idx_end=idx_stats_end)
print("\nTABLA ESTADÍSTICOS PRINCIPALES (Vista rápida):")
display(df_stats.head())

# Datos crudos
df_raw = parse_datos_crudos(lines, idx_start=idx_raw_start)
print("\nDATOS CRUDOS MEDIDOS POR EL ALS (Vista rápida):")
display(df_raw.head())


In [ ]:
# FUNCIONES PARA EXTRACCIÓN DE DATOS ESTADÍSTICOS NECESARIOS PARA SEGUIMIENTOS

# Limitar a 3 cifras significativas
def sig(x, n=3):
    try:
        return float(f"{x:.{n}g}")
    except:
        return x

# Construcción de la tabla final para seguimientos
def construir_resumen(df_stats):

    variables = df_stats.columns.tolist()
    filas = []

    for var in variables:
        try:
            # Extraer valores de df_stats
            valor_nominal = df_stats.loc["Valor nominal", var]
            xqq           = df_stats.loc["xqq", var]
            sigma         = df_stats.loc["Sigma", var]
            cp            = df_stats.loc["Cp", var]
            cpd           = df_stats.loc["Cpd", var]

            # Calcular Rango
            rango = df_stats.loc["Tolerancia superior", var] - valor_nominal

            # Calcular Desviación
            desviacion = (cp - cpd) / (cp) if cp != 0 else np.nan

            # Convertir a 3 cifras significativas cada valor de interés
            fila = [
                var,
                sig(rango),
                sig(valor_nominal),
                sig(xqq),
                sig(sigma),
                sig(cp),
                sig(cpd),
                sig(desviacion)
            ]
        # Si se encuentran valores faltantes devuelve toda la fila como "NaN"
        except KeyError:
            fila = [var] + [np.nan]*7

        filas.append(fila)

    # Se crea el DF final
    df_resumen = pd.DataFrame(
        filas,
        columns=[
            "Variables", "Rango", "Valor nominal", "Media",
            "Sigma_W", "Cp", "Cpk", "Desviación"
        ]
    )

    return df_resumen

In [ ]:
# EJECUCIÓN
df_resumen = construir_resumen(df_stats)
df_resumen

In [ ]:
# Código para generar PDF mostrando df_resumen
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

def exportar_resumen_a_pdf(df_resumen, nombre_pdf="resumen_estadisticos.pdf"):
    """
    Crea un PDF con una tabla que contiene el DataFrame df_resumen.
    Guarda el archivo en el directorio actual de Colab.
    """

    # Tamaño de la figura en función de filas y columnas
    n_filas, n_cols = df_resumen.shape
    ancho = max(8, n_cols * 1.3)      # ajusta si quieres más ancho
    alto = max(2, n_filas * 0.6 + 1)  # ajusta si quieres más alto

    fig, ax = plt.subplots(figsize=(ancho, alto))
    ax.axis("off")  # ocultar ejes

    tabla = ax.table(
        cellText=df_resumen.values,
        colLabels=df_resumen.columns,
        loc="center",
        cellLoc="center",
    )

    # Ajustes estéticos
    tabla.auto_set_font_size(False)
    tabla.set_fontsize(8)
    tabla.scale(1.1, 1.3)  # escala (ancho, alto) de las celdas

    plt.tight_layout()

    # Guardar en PDF
    with PdfPages(nombre_pdf) as pdf:
        pdf.savefig(fig, bbox_inches="tight")

    plt.close(fig)
    print(f"PDF generado: {nombre_pdf}")


In [ ]:
exportar_resumen_a_pdf(df_resumen, "Resumen_Estadisticos_ALS.pdf")


###TASKS
1. Se debe realizar el código que permita graficar las cartas de control que nos interesan a partir de los datos crudos. Como son muchos datos, se puede evaluar la opción de que la grafica tenga una barra deslizante para el desplazamiento por todo el gráfico, de manera que todos los datos se muestren de manera correcta

2. Se tiene ya los datos necesarios en df_resumen para completar la plantilla de seguimientos por máquina. En este punto, evaluar opción de incluir las columnas Cpk estimado y Rango ajustado, de forma tal que con un input se pueda ingresar el valor del rango ajustado al que se quiere llegar y calcule el Cpk estimado. Adicionalmente, evaluar la opción de generar un archivo PDF donde se muestre esta tabla final (Tal cual a la de la plantilla de seguimientos) y en la cabezara se muestra la informacion contextual (Pedido, Molde, Descripcion, Material, fecha de analisis, fecha datos analizados) y ademas, crea un input donde se pueda escribir comentarios y se muestren en la parte inferior del PDF



In [ ]:
# FUNCIONES PARA GRÁFICAS DE CONTROL

# Toma los datos crudos, clasifica para n=5 mediciones por subgrupo y realiza cálculos SPC
def procesar_datos_spc(df_raw):

    df = df_raw.copy()

    # Se separa el subgrupo con la medición en dos columnas aparte
    df[["subgrupo", "pos"]] = df["Muestra aleatoria"].str.split("/", expand=True)
    df["subgrupo"] = df["subgrupo"].astype(int)
    df["pos"] = df["pos"].astype(int)

    # Toma solo las columnas con valores númericos (Mediciones de cada variable de control)
    columnas_no_variables = ["Muestra aleatoria", "Momento", "Cantidad piezas", "subgrupo", "pos"]
    variables = [c for c in df.columns if c not in columnas_no_variables]

    # Calcular X̄ y S por subgrupo
    df_xbar = df.groupby("subgrupo")[variables].mean() # Media por subgrupo
    df_s = df.groupby("subgrupo")[variables].std(ddof=1) # Des Std por subgrupo

    xbar_prom_global = df_xbar.mean() # Cálculo de la media de todas las medias
    s_prom_global = df_s.mean() # Cálculo de la media de las desv std

    return df, df_xbar, df_s, xbar_prom_global, s_prom_global


In [ ]:
# EJECUCIÓN
df_limpio, df_xbar, df_s, xbar_prom, s_prom = procesar_datos_spc(df_raw)


In [ ]:
df_stats.index

In [ ]:
df_stats.loc["Tolerancia superior", "t4012 [s]"]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import IntSlider, interact

def graficar_carta_x_interactiva(df_xbar, xbar_prom, df_stats, variable, window=30, titulo=None):
    """
    Carta X̄ interactiva para una variable.
    """

    # ----- Datos base -----
    xbar_subgrupos = df_xbar[variable]
    subgrupos = xbar_subgrupos.index.astype(int).to_numpy()

    xbar_global = float(xbar_prom[variable])

    # Límites de especificación
    USLx = float(df_stats.loc["Tolerancia superior", variable])
    LSLx = float(df_stats.loc["Tolerancia inferior", variable])

    # Límites de control
    UCLx = float(df_stats.loc["LISx", variable])
    LCLx = float(df_stats.loc["LIIx", variable])

    # ----- Escala eje Y -----
    all_vals = np.concatenate([
        xbar_subgrupos.values,
        [USLx, LSLx, UCLx, LCLx, xbar_global]
    ])
    y_min = all_vals.min()
    y_max = all_vals.max()
    margen = 0.05 * (y_max - y_min if y_max > y_min else 1.0)
    y_min -= margen
    y_max += margen

    # ----- Ventana deslizante -----
    n = len(subgrupos)
    if window > n:
        window = n

    if titulo is None:
        titulo = (f"Carta X̄ - {variable}   "
                  f"SW={xbar_global:.3f}   "
                  f"USL={USLx:.3f}   LSL={LSLx:.3f}")

    def plot_window(start_idx: int):
        end_idx = min(start_idx + window, n)
        sg = subgrupos[start_idx:end_idx]
        vals = xbar_subgrupos.iloc[start_idx:end_idx]

        plt.figure(figsize=(12, 5))
        ax = plt.gca()

        # ----- Rellenos superiores e inferiores -----
        ax.axhspan(USLx, y_max, facecolor="red", alpha=0.08)
        ax.axhspan(y_min, LSLx, facecolor="red", alpha=0.08)

        # ----- Serie X̄ -----
        ax.plot(sg, vals, marker="o", linestyle="-", color="blue",
                label="X̄ por subgrupo")

        # ----- Media global (verde suave, punteado fino) -----
        ax.axhline(y=xbar_global, linestyle="--", linewidth=0.8,
                   color="green", label="X̄ global")

        # ----- Límites de especificación (rojo continuo) -----
        ax.axhline(y=USLx, linestyle="-", linewidth=1.2,
                   color="red", label="USLx")
        ax.axhline(y=LSLx, linestyle="-", linewidth=1.2,
                   color="red", label="LSLx")

        # ----- Límites de control (naranja, punteado fino) -----
        ax.axhline(y=UCLx, linestyle="--", linewidth=0.8,
                   color="orange", label="UCLx")
        ax.axhline(y=LCLx, linestyle="--", linewidth=0.8,
                   color="orange", label="LCLx")

        # ----- Ejes -----
        ax.set_xlabel("Subgrupos")
        ax.set_ylabel(variable)
        ax.set_title(titulo)

        # Eje X delimitado a la ventana
        ax.set_xlim(sg.min() - 0.5, sg.max() + 0.5)
        ax.set_xticks(sg)

        # Escala vertical
        ax.set_ylim(y_min, y_max)

        # Grid
        ax.grid(True, linestyle="--", alpha=0.4)

        # ----- Marco exterior negro -----
        for spine in ax.spines.values():
            spine.set_edgecolor("black")
            spine.set_linewidth(1.2)

        # ----- Leyenda fuera de la gráfica -----
        ax.legend(
            loc="upper center",
            bbox_to_anchor=(0.5, -0.20),   # Coordenadas relativas: centrada y por debajo de la gráfica
            ncol=6,                        # Número de columnas (ajústalo según tus líneas)
            frameon=False,                # Sin borde
            fontsize=9
        )
        plt.tight_layout()
        plt.show()

    max_start = max(n - window, 0)
    slider = IntSlider(
        value=0,
        min=0,
        max=max_start,
        step=1,
        description="Inicio:",
        continuous_update=False,
    )

    interact(plot_window, start_idx=slider)


In [ ]:
graficar_carta_x_interactiva(df_xbar, xbar_prom, df_stats, "t4012 [s]", window=20)


In [ ]:
# GRÁFICAS para borrar
import matplotlib.pyplot as plt
from ipywidgets import IntSlider, interact

def graficar_carta_x_interactiva(df_xbar, xbar_prom, df_stats, variable, window=50, titulo=None):
    """
    Carta X̄ interactiva para una variable.

    - df_xbar: X̄ por subgrupo (índice = subgrupo, columnas = variables)
    - xbar_prom: Serie con media global de X̄ por variable
    - df_stats: DF de estadísticos (índice incluye 'LISx' y 'LIIx')
    - variable: nombre de la columna a graficar (ej: 't4012 [s]')
    - window: cantidad de subgrupos visibles por ventana (ej. 50)
    """

    # Datos base
    xbar_subgrupos = df_xbar[variable]  # Serie X̄ para cada variable (Columnas)
    subgrupos = xbar_subgrupos.index.astype(int).to_numpy() # Se convierten los índices en un Array

    # Extraemos la medía global y los límites para graficarlos como constantes
    xbar_global = float(xbar_prom[variable])
    USLx = float(df_stats.loc["Tolerancia superior", variable])
    LSLx = float(df_stats.loc["Tolerancia inferior", variable])

    # Escala eje Y automática (con un pequeño margen)
    all_vals = np.concatenate([xbar_subgrupos.values, [USLx, LSLx, xbar_global]])
    y_min = all_vals.min()
    y_max = all_vals.max()
    margen = 0.05 * (y_max - y_min if y_max > y_min else 1.0)
    y_min -= margen
    y_max += margen

    # Para muchos subgrupos, mostraremos "ventanas" deslizables
    n = len(subgrupos)
    if window > n:
        window = n

    if titulo is None:
        titulo = f"Carta X̄ - {variable}"

    def plot_window(start_idx: int):
        end_idx = min(start_idx + window, n)
        sg = subgrupos[start_idx:end_idx]
        vals = xbar_subgrupos.iloc[start_idx:end_idx]

        plt.figure(figsize=(10, 5))
        ax = plt.gca()

        # Serie X̄
        ax.plot(sg, vals, marker="o", linestyle="-", label="X̄ por subgrupo")

        # Media global
        ax.axhline(y=xbar_global, linestyle="--", linewidth=1.5, label="Media X̄ global")

        # Límites de especificación
        ax.axhline(y=USLx, linestyle="-", linewidth=1.2, label="USLx")
        ax.axhline(y=LSLx, linestyle="-", linewidth=1.2, label="LSLx")

        # Ejes, título, etc.
        ax.set_xlabel("Subgrupos")
        ax.set_ylabel(variable)
        ax.set_title(titulo)

        # Eje X: solo los subgrupos visibles, enteros
        ax.set_xlim(sg.min() - 0.5, sg.max() + 0.5)
        ax.set_xticks(sg)

        # Eje Y centrado en el rango de datos + márgenes
        ax.set_ylim(y_min, y_max)

        ax.grid(True, linestyle="--", alpha=0.4)
        ax.legend()
        plt.tight_layout()
        plt.show()

    # Slider para moverse por los subgrupos
    max_start = max(n - window, 0)
    slider = IntSlider(
        value=0,
        min=0,
        max=max_start,
        step=1,
        description="Inicio:",
        continuous_update=False,
    )

    interact(plot_window, start_idx=slider)
